# Лабораторная работа №6

## Задание 1 
###### Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)

### Загрузка Библиотек

In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler


### Загрузка данных и предварительная обработка
###### Загрузка данных

In [3]:
N      = 100000 
N_test = 0.99992 

In [4]:
data = pd.read_csv('database.csv',low_memory=False)
subset_data = data.sample(n=N, random_state=42)
# Исключение ненужных столбцов
columns_to_drop = ['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City', 'State', 'Year', 'Month', 'Record Source']
subset_data = subset_data.drop(columns=columns_to_drop)

subset_data.dropna(inplace=True) # Удаление записей с пропущенными значениями
subset_data.head()

,Incident,Crime Type,Crime Solved,Victim Sex,Victim Age,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count
484984,1,Murder or Manslaughter,No,Male,18,White,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,1,0
608445,3,Murder or Manslaughter,No,Male,27,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Firearm,0,0
417612,1,Murder or Manslaughter,Yes,Male,42,Black,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0
372692,6,Murder or Manslaughter,No,Male,44,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0
402028,1,Murder or Manslaughter,Yes,Female,15,White,Unknown,Male,20,White,Unknown,Girlfriend,Rifle,0,0


###### Выбор признаков и целевой переменной

In [5]:
X = subset_data.drop(columns=['Crime Solved'])
y = subset_data['Crime Solved']

###### Преобразование категориальных признаков в числовые

In [6]:
X = pd.get_dummies(X)

In [7]:
# Масштабирование данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Разделение данных на тренировочную и тестовую выборки

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=N_test, random_state=42)

### Построение и оценка модели

In [9]:
# Определение модели
model = CatBoostClassifier(silent=True)

# Определение сетки гиперпараметров для поиска
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'iterations': [100, 200, 300]
}

# Настройка GridSearchCV
grid_search = GridSearchCV(
    estimator=model, 
    param_grid=param_grid, 
    cv=3, 
    scoring='accuracy', 
    n_jobs=-1
)

In [10]:
%%time
# Обучение модели
grid_search.fit(X_train, y_train)

CPU times: total: 969 ms
Wall time: 11.8 s


GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x000001F3DE413050>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10], 'iterations': [100, 200, 300],
                         'learning_rate': [0.01, 0.1, 0.2]},
             scoring='accuracy')

In [11]:
# Лучшая модель
best_model = grid_search.best_estimator_
print("Наилучшие гиперпараметры:")
print(grid_search.best_params_)

Наилучшие гиперпараметры:
{'depth': 6, 'iterations': 100, 'learning_rate': 0.01}


In [12]:
%%time

# Предсказание на тестовой выборке
y_pred = best_model.predict(X_test)

# Оценка модели

print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred))

print("Точность:", accuracy_score(y_test, y_pred))

print("Отчет о классификации:")
print(classification_report(y_test, y_pred))

Матрица ошибок:
 [[29634    35]
 [ 2622 67701]]
Точность: 0.9734278742299384
Отчет о классификации:
              precision    recall  f1-score   support

          No       0.92      1.00      0.96     29669
         Yes       1.00      0.96      0.98     70323

    accuracy                           0.97     99992
   macro avg       0.96      0.98      0.97     99992
weighted avg       0.98      0.97      0.97     99992

CPU times: total: 9.75 s
Wall time: 9.75 s
